Setelah membuat dataframe yang berisi seluruh kecamatan di kabupaten Banyumas, kita akan melanjutkan dengan mendapatkan nilai curah hujan (tp) hasil expertise forecaster. Nilai curah hujan ini akan diagregasi harian dan akan dijadikan sebagai komponen Hazard

In [1]:
import pandas as pd

In [3]:
data_fct = "/content/forecast_banyumas.csv"
df = pd.read_csv(data_fct,parse_dates=['datetime'])

df

,datetime,t,tcc,tp,weather,weather_desc,weather_desc_en,wd_deg,wd,wd_to,...,adm1,adm2,adm3,provinsi,kotkab,kecamatan,lon,lat,timezone,type
0,2024-11-16 12:00:00+00:00,25,100,0.2,3,Berawan,Mostly Cloudy,49,NE,SW,...,33,33.02,33.02.01,Jawa Tengah,Banyumas,Lumbir,108.975911,-7.465557,700,adm3
1,2024-11-16 15:00:00+00:00,24,98,0.0,3,Berawan,Mostly Cloudy,33,N,S,...,33,33.02,33.02.01,Jawa Tengah,Banyumas,Lumbir,108.975911,-7.465557,700,adm3
2,2024-11-16 18:00:00+00:00,23,99,0.1,3,Berawan,Mostly Cloudy,322,NW,SE,...,33,33.02,33.02.01,Jawa Tengah,Banyumas,Lumbir,108.975911,-7.465557,700,adm3
3,2024-11-16 21:00:00+00:00,23,100,0.0,3,Berawan,Mostly Cloudy,357,NW,SE,...,33,33.02,33.02.01,Jawa Tengah,Banyumas,Lumbir,108.975911,-7.465557,700,adm3
4,2024-11-17 00:00:00+00:00,25,100,0.0,3,Berawan,Mostly Cloudy,37,N,S,...,33,33.02,33.02.01,Jawa Tengah,Banyumas,Lumbir,108.975911,-7.465557,700,adm3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,2024-11-19 09:00:00+00:00,26,100,6.4,61,Hujan Ringan,Light Rain,247,SW,NE,...,33,33.02,33.02.27,Jawa Tengah,Banyumas,Purwokerto Utara,109.239865,-7.402663,700,adm3
752,2024-11-19 12:00:00+00:00,24,100,0.5,3,Berawan,Mostly Cloudy,91,E,W,...,33,33.02,33.02.27,Jawa Tengah,Banyumas,Purwokerto Utara,109.239865,-7.402663,700,adm3
753,2024-11-19 15:00:00+00:00,24,94,0.0,3,Berawan,Mostly Cloudy,91,E,W,...,33,33.02,33.02.27,Jawa Tengah,Banyumas,Purwokerto Utara,109.239865,-7.402663,700,adm3
754,2024-11-19 18:00:00+00:00,23,32,0.0,1,Cerah,Sunny,2,N,S,...,33,33.02,33.02.27,Jawa Tengah,Banyumas,Purwokerto Utara,109.239865,-7.402663,700,adm3


Selanjutnya kita akan mengambil kolom yang akan digunakan.

In [10]:
df_sub = df[['datetime','tp','adm3','kecamatan',]]
# shift datetime to -3 hours
df_sub['datetime'] = df_sub['datetime'] - pd.Timedelta(hours=3)
df_sub.set_index('datetime',inplace=True)
df_sub


<ipython-input-10-81cfe740663f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['datetime'] = df_sub['datetime'] - pd.Timedelta(hours=3)


,tp,adm3,kecamatan
datetime,,,
2024-11-16 09:00:00+00:00,0.2,33.02.01,Lumbir
2024-11-16 12:00:00+00:00,0.0,33.02.01,Lumbir
2024-11-16 15:00:00+00:00,0.1,33.02.01,Lumbir
2024-11-16 18:00:00+00:00,0.0,33.02.01,Lumbir
2024-11-16 21:00:00+00:00,0.0,33.02.01,Lumbir
...,...,...,...
2024-11-19 06:00:00+00:00,6.4,33.02.27,Purwokerto Utara
2024-11-19 09:00:00+00:00,0.5,33.02.27,Purwokerto Utara
2024-11-19 12:00:00+00:00,0.0,33.02.27,Purwokerto Utara


Mengubah data per 3 jam menjadi harian

In [12]:
resampled_data = (
    df_sub
    .groupby('kecamatan')
    .resample('D')
    .agg({'tp': 'sum','adm3':'first'})
    .reset_index()
)
resampled_data

,kecamatan,datetime,tp,adm3
0,Ajibarang,2024-11-16 00:00:00+00:00,3.4,33.02.14
1,Ajibarang,2024-11-17 00:00:00+00:00,39.8,33.02.14
2,Ajibarang,2024-11-18 00:00:00+00:00,35.1,33.02.14
3,Ajibarang,2024-11-19 00:00:00+00:00,17.3,33.02.14
4,Banyumas,2024-11-16 00:00:00+00:00,5.9,33.02.11
...,...,...,...,...
103,Tambak,2024-11-19 00:00:00+00:00,21.4,33.02.08
104,Wangon,2024-11-16 00:00:00+00:00,0.0,33.02.02
105,Wangon,2024-11-17 00:00:00+00:00,27.0,33.02.02
106,Wangon,2024-11-18 00:00:00+00:00,24.8,33.02.02


Simpan dalam format CSV

In [13]:
resampled_data.to_csv("forecast_tp_banyumas.csv",sep=',',index=False)